# Analyse topological divergences and the baselines for Lyapunov estimation

In [36]:
import pandas as pd
import os
import pickle

RESULTS_DIR = 'outputs/data/divergence_results'
pkl_files = [f for f in os.listdir(RESULTS_DIR) if f.endswith(('_500.pkl', '_750.pkl', '_1000.pkl'))]

data_frames = []

for pkl_file in pkl_files:
    # Load the dictionary from the .pkl file
    with open(f'{RESULTS_DIR}/{pkl_file}', 'rb') as file:
        data_dict = pickle.load(file)
    
    # Convert dictionary into a Pandas DataFrame
    data_frame = pd.DataFrame(list(data_dict.items()), columns=['Keys', 'Values'])
    
    # Split the tuple keys into separate columns
    data_frame[['measure', 'system', 'length']] = pd.DataFrame(data_frame['Keys'].tolist(), index=data_frame.index)

    # Drop the original keys column
    data_frame = data_frame.drop(['Keys'], axis=1)
    
    # Add the dataframe to the list
    data_frames.append(data_frame)

# Concatenate the data frames
merged_data = pd.concat(data_frames)

# Split values dictionaries into separate columns and drop original column
merged_data = pd.concat([merged_data.drop(['Values'], axis=1), merged_data['Values'].apply(pd.Series)], axis=1)

# Further split the two correlation columns into the coefficient and pvalues
merged_data[['spearmanr_corr', 'spearmanr_pval']] = merged_data['spearmanr'].apply(pd.Series)
merged_data[['pos_spearmanr_corr', 'pos_spearmanr_pval']] = merged_data['pos_spearmanr'].apply(pd.Series)
merged_data = merged_data.drop(columns=['spearmanr', 'pos_spearmanr'])

merged_data['system'] = merged_data['system'].replace("Hénon", "henon")
merged_data['system'] = merged_data['system'].replace("Ikeda", "ikeda")
merged_data['system'] = merged_data['system'].replace("Logistic", "logistic")
merged_data['system'] = merged_data['system'].replace("Tinkerbell", "tinkerbell")

In [37]:
merged_data

,measure,system,length,classification_f1,regression_neg_mean_absolute,pos_regression_neg_mean_absolute,spearmanr_corr,spearmanr_pval,pos_spearmanr_corr,pos_spearmanr_pval
0,cophenetic divergence,henon,500,0.854496,-0.283391,-0.531585,0.736482,1.396494e-86,0.746799,3.699274e-43
1,cophenetic_reverse divergence,henon,500,0.838534,-0.359803,-0.517885,0.798787,5.628399e-112,0.831906,1.592406e-61
2,interleaving_0.5_None divergence,henon,500,0.815870,-0.328161,-0.622283,0.795286,2.552484e-110,0.606278,5.620223e-25
3,interleaving_length_0.5_None divergence,henon,500,0.832790,-0.306642,-0.618956,0.825353,1.022454e-125,0.689579,1.681493e-34
4,interleaving_edge_0.5_None divergence,henon,500,0.804656,-0.337078,-0.573068,0.790853,2.875452e-108,0.699864,6.632495e-36
...,...,...,...,...,...,...,...,...,...,...
7,Eckmann,tinkerbell,1000,0.972552,-0.670279,-0.726497,0.626371,7.697848e-56,0.512385,3.190631e-31
8,Kantz,tinkerbell,1000,0.960502,-0.621571,-0.574721,0.761935,5.239992e-96,0.703582,7.161184e-68
9,Rosenstein,ikeda,1000,0.954999,-0.172387,-0.913922,0.786961,1.657966e-106,-0.549278,1.490127e-20
10,Eckmann,ikeda,1000,0.950663,-0.250922,-0.886572,0.037937,3.972846e-01,-0.725559,5.386940e-41


## Classic Algorithms

In [78]:
df = merged_data
filtered_df = df[df['length'] == 1000]
# filtered_df = df
pivot_table = pd.pivot_table(filtered_df, values='regression_neg_mean_absolute', index=['measure'], columns=['system'], aggfunc='mean')

In [79]:
display(pivot_table)

system,henon,ikeda,logistic,tinkerbell
measure,,,,
$\Delta$VGA ($L_1$),-0.393277,-0.442392,-0.398594,-0.919487
$\Delta$VGA ($W_1$),-0.332886,-0.480306,-0.449101,-1.416052
$k$-NN graph $E'(D)$,-0.257480,-0.347617,-0.320957,-0.724081
$k$-NN graph $M(D)$,-0.234444,-0.306778,-0.372151,-0.604457
$k$-NN graph $P(D)$,-0.262947,-0.334764,-0.320332,-0.803502
Eckmann,-0.189888,-0.250922,-0.470324,-0.670279
Kantz,-0.109678,-0.162642,-0.324439,-0.621571
Ordinal graph $E'(D)$,-0.252113,-0.393685,-0.335061,-0.812266
Ordinal graph $M(D)$,-0.258632,-0.351237,-0.327848,-0.816749


In [80]:
latex_table = pivot_table.style.to_latex()

In [81]:
def highlight_max(s):
    is_max = s == s.max()
    return ['font-weight: bold' if v else '' for v in is_max]

# Apply the styling function to the pivot table and display it
styled = pivot_table.style.apply(highlight_max)
display(styled)


system,henon,ikeda,logistic,tinkerbell
measure,,,,
$\Delta$VGA ($L_1$),-0.393277,-0.442392,-0.398594,-0.919487
$\Delta$VGA ($W_1$),-0.332886,-0.480306,-0.449101,-1.416052
$k$-NN graph $E'(D)$,-0.257480,-0.347617,-0.320957,-0.724081
$k$-NN graph $M(D)$,-0.234444,-0.306778,-0.372151,-0.604457
$k$-NN graph $P(D)$,-0.262947,-0.334764,-0.320332,-0.803502
Eckmann,-0.189888,-0.250922,-0.470324,-0.670279
Kantz,-0.109678,-0.162642,-0.324439,-0.621571
Ordinal graph $E'(D)$,-0.252113,-0.393685,-0.335061,-0.812266
Ordinal graph $M(D)$,-0.258632,-0.351237,-0.327848,-0.816749


In [64]:
latex_styled = styled.to_latex()

In [65]:
print(latex_styled)

\begin{tabular}{lrrrr}
system & henon & ikeda & logistic & tinkerbell \\
measure &  &  &  &  \\
$\Delta$VGA ($L_1$) & 0.434649 & 0.049984 & 0.694473 & 0.187999 \\
$\Delta$VGA ($W_1$) & 0.368338 & 0.128813 & 0.852349 & 0.033496 \\
$k$-NN graph $E'(D)$ & -0.044191 & -0.144560 & -0.605705 & 0.623138 \\
$k$-NN graph $M(D)$ & 0.594765 & 0.278052 & 0.186924 & 0.760750 \\
$k$-NN graph $P(D)$ & 0.401049 & 0.583349 & 0.255127 & 0.620634 \\
Eckmann & 0.468816 & -0.725559 & -0.178014 & 0.512385 \\
Kantz & 0.736105 & 0.145367 & 0.723014 & 0.703582 \\
Ordinal graph $E'(D)$ & -0.093603 & 0.123760 & -0.019531 & 0.160881 \\
Ordinal graph $M(D)$ & 0.111091 & 0.251806 & 0.112793 & -0.159029 \\
Ordinal graph $P(D)$ & 0.624606 & 0.049558 & 0.670583 & -0.055606 \\
Rosenstein & 0.332005 & -0.549278 & -0.378957 & \font-weightbold 0.905659 \\
bottleneck divergence & 0.049107 & 0.361167 & -0.151166 & 0.029967 \\
cophenetic & 0.772486 & 0.119886 & 0.209931 & 0.796909 \\
cophenetic divergence & 0.823282 & 0.8851